In [1]:
import numpy as np
import pandas as pd
import gensim
from gensim.models import Doc2Vec

C:\Users\Asus\Anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
df=pd.read_csv("df_c.txt")

In [3]:
df.head()

,Unnamed: 0,Title,Release Year,Plot,ID
0,30589,The Magnificent Seven,1960,A Mexican village is periodically raided for f...,162350
1,9586,G.I. Blues,1960,U.S. Army Specialist 5 Tulsa McLean is a tan...,3602
2,9567,Hell Is a City,1960,Committed but seen-it-all police inspector Mar...,130422
3,13144,Spartacus,1960,"In the first century BC, the Roman Republic ha...",2728
4,31524,The Angel Wore Red,1960,A clergyman travels to Spain to join the Loyal...,124695


In [6]:
df.ID.nunique()

16239

There are dublicated rows in terms of title and ID. To investigate them I create a subset dataset that only consists of that rows. Most of them have different release year value, so I keep the ones that has the same release year value with the movielens dataset.

In [4]:
df_dub=df.loc[df.ID.duplicated(keep=False)]

In [5]:
df_dub.sort_values(by="ID")

,Unnamed: 0,Title,Release Year,Plot,ID
4182,35983,Heat,1986,A woman in a Las Vegas bar is annoyed by a fli...,6
12000,20158,Heat,2006,After three years a sailor Aleksei comes back...,6
1775,20458,Heat,1972,Joey Davis is an unemployed former child star...,6
6831,14626,Heat,1995,Career criminal Neil McCauley and his crew—C...,6
12744,34234,Persuasion,2007,Persuasion tells the story of Anne Elliot and ...,28
...,...,...,...,...,...
15343,18401,Shakti,2011,Aishwarya is the daughter of central union ho...,207461
1216,3017,The Chairman,1969,A US agent is sent to communist China in order...,208100
570,14412,The Chairman,1964,"After World War II ends, soldier Yegor Trubnik...",208100
16966,52630,Lens,2016,Aravind’s (Jayaprakash Radhakrishnan) growing ...,208219


In [85]:
df_dub.rename(columns={"isin":"isinml"},inplace=True)

C:\Users\Asus\AppData\Roaming\Python\Python37\site-packages\pandas\core\frame.py:4449: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [101]:
ml_movies=pd.read_csv("movies.csv")

In [91]:
ml_movies=ml_movies.loc[ml_movies.movieId.isin(df.ID.unique())]

In [107]:
ml_movies.title=ml_movies.title.map(lambda x: x[:-1] if x.endswith(" ") else x)

In [108]:
ml_movies=ml_movies.loc[ml_movies.title.str.endswith(")")]

In [109]:
ml_movies["Release Year"]=ml_movies.title.str[-5:-1]

In [110]:
k=ml_movies["Release Year"].unique().tolist()

In [92]:
ml_movies.title.str.endswith(")").sum()

16198

In [112]:
k.remove("ions")
k.remove("piel")
k.remove("wing")
k.remove("983)")

In [115]:
ml_movies=ml_movies.loc[ml_movies["Release Year"].isin(k)]

In [116]:
ml_movies["Release Year"]=ml_movies["Release Year"].astype(np.int16)

In [118]:
ml_movies

,movieId,title,genres,Release Year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II (1995),Comedy,1995
...,...,...,...,...
62418,209157,We (2018),Drama,2018
62419,209159,Window of the Soul (2001),Documentary,2001
62420,209163,Bad Poems (2018),Comedy|Drama,2018
62421,209169,A Girl Thing (2001),(no genres listed),2001


In [119]:
def isinml(row):
    return (row["Release Year"]==ml_movies.loc[ml_movies.movieId==row["ID"]]["Release Year"]).values
    

In [120]:
df_dub["isin"]=df_dub.apply(lambda row: isinml(row),axis=1)

C:\Users\Asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [125]:
df_dub_k=df_dub.loc[df_dub.mlisin.values==True]

C:\Users\Asus\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  """Entry point for launching an IPython kernel.


In [127]:
df_dub_k

,Unnamed: 0,Title,Release Year,Plot,ID,mlisin
3,13144,Spartacus,1960,"In the first century BC, the Roman Republic ha...",2728,[True]
30,27220,Exodus,1960,The film is based on the events that happened ...,5799,[True]
63,25459,Kidnapped,1960,"In eighteenth century Scotland, young David Ba...",2079,[True]
85,28365,Tormented,1960,"Jazz pianist Tom Stewart , who lives on a Cape...",7245,[True]
102,27608,Late Autumn,1960,Three middle-aged friends and former college m...,68956,[True]
...,...,...,...,...,...,...
17559,47224,The Discovery,2017,The film opens with an interview with Thomas H...,170629,[True]
17576,47222,The Zookeeper's Wife,2017,Dr. Jan Żabiński (Johan Heldenbergh) and his w...,170645,[True]
17577,47242,King Arthur: Legend of the Sword,2017,"Mordred, an iron-fisted warlock, and his armie...",168344,[True]
17580,47265,Baby Driver,2017,"Baby is a getaway driver in Atlanta, Georgia. ...",171763,[True]


In [123]:
df_dub.rename(columns={"isin":"mlisin"},inplace=True)

C:\Users\Asus\AppData\Roaming\Python\Python37\site-packages\pandas\core\frame.py:4449: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [130]:
df_dub_k.drop(columns=["Unnamed: 0","mlisin"],inplace=True)

C:\Users\Asus\AppData\Roaming\Python\Python37\site-packages\pandas\core\frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [131]:
df_dub_k

,Title,Release Year,Plot,ID
3,Spartacus,1960,"In the first century BC, the Roman Republic ha...",2728
30,Exodus,1960,The film is based on the events that happened ...,5799
63,Kidnapped,1960,"In eighteenth century Scotland, young David Ba...",2079
85,Tormented,1960,"Jazz pianist Tom Stewart , who lives on a Cape...",7245
102,Late Autumn,1960,Three middle-aged friends and former college m...,68956
...,...,...,...,...
17559,The Discovery,2017,The film opens with an interview with Thomas H...,170629
17576,The Zookeeper's Wife,2017,Dr. Jan Żabiński (Johan Heldenbergh) and his w...,170645
17577,King Arthur: Legend of the Sword,2017,"Mordred, an iron-fisted warlock, and his armie...",168344
17580,Baby Driver,2017,"Baby is a getaway driver in Atlanta, Georgia. ...",171763


In [132]:
df.shape

(17613, 5)

In [133]:
df.drop_duplicates(subset=["ID"],keep=False,inplace=True)

In [135]:
df.head()

,Unnamed: 0,Title,Release Year,Plot,ID
0,30589,The Magnificent Seven,1960,A Mexican village is periodically raided for f...,162350
1,9586,G.I. Blues,1960,U.S. Army Specialist 5 Tulsa McLean is a tan...,3602
2,9567,Hell Is a City,1960,Committed but seen-it-all police inspector Mar...,130422
4,31524,The Angel Wore Red,1960,A clergyman travels to Spain to join the Loyal...,124695
7,41293,The Beatniks,1960,Eddy Crane is the leader of a gang that robs s...,149666


In [134]:
df.shape

(15078, 5)

In [136]:
df.drop(columns=["Unnamed: 0"],inplace=True)

In [137]:
df=pd.concat([df,df_dub_k])

In [139]:
df.head()

,Title,Release Year,Plot,ID
0,The Magnificent Seven,1960,A Mexican village is periodically raided for f...,162350
1,G.I. Blues,1960,U.S. Army Specialist 5 Tulsa McLean is a tan...,3602
2,Hell Is a City,1960,Committed but seen-it-all police inspector Mar...,130422
4,The Angel Wore Red,1960,A clergyman travels to Spain to join the Loyal...,124695
7,The Beatniks,1960,Eddy Crane is the leader of a gang that robs s...,149666


In [138]:
df.shape

(16234, 4)

In [103]:
df.shape

(16234, 5)

In [165]:
df.ID.duplicated().sum()

232

In [169]:
df_dub2=df.loc[df.ID.duplicated(keep=False)]

df_dub2.sort_values(by="ID",inplace=True)

df_dub2

,Title,Release Year,Plot,ID
380,Bhaji on the Beach,1993,A community group of British women (mostly Pun...,568
381,Bhaji on the Beach,1993,A community group of British women (mostly Pun...,568
1284,Soldier,1998,Captain Vijay Malhotra attempts to defend him...,2322
1285,Soldier,1998,"In the near future, as part of a new military ...",2322
1390,Gloria,1999,Gloria is set against the backdrop of a rural ...,2479
...,...,...,...,...
15586,Kanthaswamy,2009,"The film's plot revolves around Kanthasamy , w...",192073
15624,Yavarum Nalam,2009,"Manohar, an upper-middle class Indian, moves ...",192481
15625,Yavarum Nalam,2009,"Manohar moves into a new apartment, 13B, on th...",192481
15901,Out of Inferno,2013,The film stars two estranged brothers in Guang...,199053


these are the rows for same exact movies so I drop one of them.

In [170]:
df.drop_duplicates(subset=["ID"],inplace=True)

In [140]:
df=df.sort_values(by=["ID"])

In [ ]:
df.reset_index(inplace=True, drop=True)

training doc2vec model 

In [172]:
from multiprocessing import cpu_count
cpus = cpu_count()

def read_corpus():
    for i,plot in enumerate(df.Plot):
        yield gensim.models.doc2vec.TaggedDocument(gensim.utils.simple_preprocess(plot), [i])

train_corpus = list(read_corpus())

In [173]:
len(train_corpus)

16002

In [174]:
model = Doc2Vec(vector_size=150, window=5, 
                negative=5, min_count=5, workers=cpus, epochs=15)
model.build_vocab(train_corpus)

model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

gensim.models.Doc2Vec.save(model,"doc2vec_n.py")

In [175]:
sim_matrix=np.empty((16002,16002))
for i in range(16002):
    inferred_vector = model.infer_vector(train_corpus[i].words)
    sims = model.dv.most_similar([inferred_vector], topn=16002)
    sim_list=[]
    for k,v in sorted(sims):
        sim_list.append(v)
    sim_matrix[i]=sim_list
    

In [179]:
np.save('simmatrix.npy', sim_matrix)

np.savetxt('simmatrix.txt', sim_matrix)

In [190]:
df.to_csv("df_plots.txt")